In [10]:
import cv2
import numpy as np
img = cv2.imread("/home/anastasia/Рабочий стол/segment1/in/2",  cv2.IMREAD_GRAYSCALE)
#img = cv2.bitwise_not(img)
##cv2.imshow("image",img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [11]:
def wavefrontRegionGrowing(img):
#    if (img.type != cv2.CV_32SC1):
#        print("image is not cv_32sc1")
#        return
    #spreading_map = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    #cv2.imshow("image",img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    
    
    spreading_map = img.copy()
    finished = False
    counter = 0
    while (finished == False):
        counter = counter + 1
        print ("step ", counter, " of region growing")
        if (counter == 100):
            break
        spreading_map = img.copy()
        finished = True
        #print(spreading_map.shape[0], spreading_map.shape[1])
        for row in range(1, spreading_map.shape[0] - 1):
            for column in range(1, spreading_map.shape[1] - 1):
                if (spreading_map[row][column] >= 255):
                    set_value = False
                    for row_counter in range(-1, 2, 1):
                        for column_counter in range(-1, 2, 1):
                            if (set_value is True):
                                break
                            value = img[row + row_counter][column+column_counter]
                            if (value != 0) and (value < 255):
                                #print("updated for row ", row, " and column ", column)
                                spreading_map[row][column] = value
                                set_value = True
                                finished = False
        #cv2.imshow("image",img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #if ((counter % 10) == 0):
        #    cv2.imshow("image",img)
        #    cv2.waitKey(0)
        #    cv2.destroyAllWindows()
        img = spreading_map.copy()
        
    return img

In [12]:
def segmentmap(file, map_resolution,room_area_factor_lower_limit, room_area_factor_upper_limit):
    
    file = cv2.threshold(file, 240, 255, cv2.THRESH_BINARY)[1]
    temp_map = file.copy()
    for i in range (20):
        erode_kernel = cv2.getStructuringElement(cv2.MORPH_ERODE, (3,3))
        #gray=cv2.threshold(file,200,255,cv2.THRESH_BINARY)[1]
        temp_map= cv2.erode(temp_map, erode_kernel)
        ##eroded=cv2.cvtColor(eroded,cv2.COLOR_BGR2GRAY)

        contour_map = temp_map.copy()
        contour_map, temp_contours, hierarchy = cv2.findContours(contour_map, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        hierarchy = hierarchy[0]
        saved_contours = []
        if (len(temp_contours) != 0):
            for curr_contour in range (len(temp_contours)):
                if (hierarchy[curr_contour][3] == -1):
                    room_area = map_resolution * map_resolution * cv2.contourArea(temp_contours[curr_contour])
                    for hole in range (len(temp_contours)):
                        if (hierarchy[hole][3] == curr_contour):
                            room_area -= map_resolution * map_resolution * cv2.contourArea(temp_contours[hole])
                if ((room_area_factor_lower_limit < room_area) and (room_area < room_area_factor_upper_limit)):
                   ## print ("room area:", room_area)
                    saved_contours.append(temp_contours[curr_contour])
                    cv2.drawContours(temp_map, temp_contours, curr_contour, cv2.COLOR_BGR2HSV)
    print ("number of rooms found:", len(saved_contours))
    
    segmented_map = cv2.cvtColor(file, cv2.COLOR_GRAY2RGB)
    
    already_used_colors = []
    for i in range (len(saved_contours)):
        drawn = False
        draw_counter = 0
        while (not drawn):
            draw_counter+=1
            color1 = (list(np.random.choice(range(256), size=3)))  
            fill_color =[int(color1[0]), int(color1[1]), int(color1[2])] 
            if ((fill_color not in already_used_colors) or draw_counter > 250):
                cv2.drawContours(segmented_map, saved_contours, i, fill_color, cv2.FILLED)
                already_used_colors.append(fill_color)
                drawn = True
    cv2.imshow("image",segmented_map)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    segmented_map = cv2.cvtColor(segmented_map, cv2.COLOR_RGB2GRAY)
    
    for row in range(file.shape[0]):
        for col in range(file.shape[1]):
            if file[row][col] is 0:
                segmented_map[row][col] = 0

    #cv2.imwrite("/home/anastasia/Рабочий стол/444.png", segmented_map)
                
    segmented_map = wavefrontRegionGrowing(segmented_map)  
    
    #cv2.imshow("image",segmented_map)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    cv2.imwrite("/home/anastasia/Рабочий стол/segment1/out/2.png", segmented_map)
   
    

In [13]:
segmentmap(img, 0.05, 0.08, 470)


number of rooms found: 16
step  1  of region growing
step  2  of region growing
step  3  of region growing
step  4  of region growing
step  5  of region growing
step  6  of region growing
step  7  of region growing
step  8  of region growing
step  9  of region growing
step  10  of region growing
step  11  of region growing
step  12  of region growing
step  13  of region growing
step  14  of region growing
step  15  of region growing
step  16  of region growing
step  17  of region growing
step  18  of region growing
step  19  of region growing
step  20  of region growing
step  21  of region growing
step  22  of region growing
step  23  of region growing
step  24  of region growing
step  25  of region growing
step  26  of region growing
step  27  of region growing
step  28  of region growing
step  29  of region growing
step  30  of region growing
step  31  of region growing
step  32  of region growing
step  33  of region growing
step  34  of region growing
step  35  of region growing
ste